# Table of Contents
 <p><div class="lev1"><a href="#Derive-scaling"><span class="toc-item-num">1&nbsp;&nbsp;</span>Derive scaling</a></div><div class="lev2"><a href="#Compute-statistics"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Compute statistics</a></div><div class="lev3"><a href="#1D-PDFs"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>1D PDFs</a></div><div class="lev3"><a href="#Decompose-variance-into-its-contributions-from-qvstar-and-omega"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Decompose variance into its contributions from qvstar and omega</a></div>

In [1]:
%load_ext autoreload
%matplotlib inline

In [2]:
%autoreload 2

print("-- loading modules")
print()

import numpy as np
import numpy.ma as ma
import dask.array as da
import matplotlib
# matplotlib.use("PDF")
import matplotlib.pyplot as plt
import datetime as dt
import time
import sys,os,glob
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LogNorm

## Add own library to path
workdir = os.getcwd()
thismodule = sys.modules[__name__]
moduledir = os.path.join(os.path.dirname(workdir),'functions')
sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])

## Load own libraries
from environmentAndDirectories import *
from importingData import *
from scalingApproximations import *
from slicingAndSubsetting import *
from thermoConstants import L_v,R_v
from plotMaps import *
from plot1DInvLog import *
from plot2D import *
from statisticalDistributions import *
from outputResults import *

## Graphical parameters
plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

-- loading modules

Own modules available: ['CAMsettings', 'daskOptions', 'environmentAndDirectories', 'importingData', 'outputResults', 'physicalConstants', 'plot1DInvLog', 'plot2D', 'plotMaps', 'scalingApproximations', 'slicingAndSubsetting', 'statisticalDistributions', 'thermoConstants', 'thermoFunctions']


In [3]:
%%time
print("-- load environment and variables")
print()

omega_id = 'OMEGA'
pr_id = 'PRECT'
ts_id = 'TS'
ta_id = 'T'
relhum_id = 'RELHUM'
ps_id = 'PS'

all_varnames = 'omega','pr','ts','ta','relhum','ps','pres'

ref_time_stride = '1h'
ref_resolution = '1dx'

print("- load data for the following options:")
compset = 'FSPCAMm_AMIP'
experiment = 'piControl'
member = 'r1i1p1'
subset = 'tropics'
daskarray = False
tracktime = True
# dates = ('185005010100','185005020000')
dates = ('185005010100','185105010000')
handle = 'h0'
plotAll2dPDFs = True
bootstrap = True

-- load environment and variables

- load data for the following options:
CPU times: user 384 µs, sys: 350 µs, total: 734 µs
Wall time: 468 µs


In [4]:
if tracktime:
    t0 = time.time()
    t_loops = []

In [5]:
for v in ['compset','experiment','member','subset','ref_time_stride','ref_resolution',\
          'daskarray','dates']:
    print("%s:"%v,getattr(thismodule,v))

print()
omega_init,pr_init,ts_init,ta_init,relhum_init,ps_init = \
    getValues([omega_id,pr_id,ts_id,ta_id,relhum_id,ps_id],
              compset,subset,experiment,ref_time_stride,ref_resolution,
              daskarray=daskarray,dates=dates,handle=handle)
pr_init *= rho_l    # convert from m/s to kg/m2/s

print()
print("get inputpaths")

historyFilesSettings = getCAMHistoryFilesSettings()
inputdir, inputdir_processed_day, inputdir_processed_1hr, inputdir_results, inputdir_fx = \
    getInputDirectories(compset,experiment)

print("compute pressure levels")
    
input_lev_file = os.path.join(inputdir_fx,'lev_fx_CESM111-SPCAM20_allExperiments_r0i0p0.nc')
computeP = getPressureCoordinateFunction(input_lev_file)
pres_init = computeP(ps_init)

cn = getArrayType(pres_init)

compset: FSPCAMm_AMIP
experiment: piControl
member: r1i1p1
subset: tropics
ref_time_stride: 1h
ref_resolution: 1dx
daskarray: False
dates: ('185005010100', '185105010000')

Importing OMEGA, PRECT, TS, T, RELHUM, PS from 24 history files between 1850-05-01-03600 and 1850-05-02-00000

get inputpaths
compute pressure levels


In [6]:
if tracktime:
    t1 = time.time()
    t_loops.append(t1)
    print()
    print("! time spent to load the data:",time2str(t1-t0))



! time spent to load the data: 00:00:01


In [7]:
#---- single scale ----#
# start comment
time_stride = '1h'
resolution = '3dx'
# stop comment

#---- multiple scales ----#
# # start uncomment
# time_strides = '1h','3h','6h','12h','1d','2d','4d','8d'
# resolutions = '1dx','2dx','3dx','4dx','5dx','6dx','7dx','8dx','9dx'
# for time_stride in time_strides:
#     for resolution in resolutions:
# # stop uncomment

#++# start indentation for multiple scales

print()
print("-----------------------------------------------------------------")
print("!          Start analysis for time_stride {:4}                  !".format(time_stride))
print("!                          and resolution {:4}                  !".format(resolution))
print("-----------------------------------------------------------------")
print()


-----------------------------------------------------------------
!          Start analysis for time_stride 1h                    !
!                          and resolution 3dx                   !
-----------------------------------------------------------------



In [8]:
print("convert data to target resolutions")

tc0 = time.time()

for varname in all_varnames:
    
    tc1 = time.time()
    setattr(thismodule,varname,
            coarsenTimeStride(coarsenResolution(getattr(thismodule,"%s_init"%varname),
                                                resolution),
                              time_stride))
    print("- %s:"%varname,time.time()-tc1,'s')
print("- all variables:",time.time()-tc0,'s')

qvstar = saturationSpecificHumidity(pres=pres,temp=ta)

convert data to target resolutions
- omega: 0.02851414680480957 s
- pr: 0.001260995864868164 s
- ts: 0.0016248226165771484 s
- ta: 0.028811931610107422 s
- relhum: 0.024110078811645508 s
- ps: 0.0012328624725341797 s
- pres: 0.01790475845336914 s
- all variables: 0.10762810707092285 s


In [9]:
print()
print("-- define output paths")
print()

# Figure output directory
figdir = os.path.join(os.path.dirname(workdir),'figures','omega500tsps',compset,experiment,member,subset,\
                      time_stride,resolution)
os.makedirs(figdir,exist_ok=True)

# Results output directory
resultdir = os.path.join(os.path.dirname(workdir),'results','omega500tsps',compset,experiment,member,subset,\
                      time_stride,resolution)
os.makedirs(resultdir,exist_ok=True)

print()


-- define output paths




# Derive scaling

In [10]:
print()
print("-- compute (percentile-wise and pointwise) scaling approximation")
print()


-- compute (percentile-wise and pointwise) scaling approximation



In [11]:
%%time
print("Compute ranks and ranks locations")

n_pts = pr.size
Q_IL = getInvLogRanks(n_pts,n_pts_per_bin=1,fill_last_decade=True)
i_Q = indexOfRank(99.9,Q_IL)
iQ_slice = slice(i_Q-5,i_Q+5)
ranks, centers, bins = computePercentilesAndBinsFromRanks(pr.flatten(),Q_IL)
iQ_min = 8
iQ_max = min(len(Q_IL),41)
# iQ_max = min(len(Q_IL),iQ_slice.stop)

# Define reference percentiles and compute percentiles
targetranks = Q_IL[iQ_min:iQ_max]
ranks_ref = Q_IL[:iQ_max]
percentiles = adjustRanks(centers,ranks,ranks_ref)

# Suffix to save figures and results
output_suffix = 'Q%d-Q%d_%s'%(iQ_min,iQ_max,'-'.join(dates))

rank_locations= {}
for rank in ranks_ref:
    rank_id  = "%2.4f"%rank
    print(rank_id,end=' ')
    rank_locations[rank_id] = getRankLocations(rank,pr,ranks,bins,rank_locations)
print()

Compute ranks and ranks locations
0.0000 20.5672 36.9043 49.8813 60.1893 68.3772 74.8811 80.0474 84.1511 87.4107 90.0000 92.0567 93.6904 94.9881 96.0189 96.8377 97.4881 98.0047 98.4151 98.7411 99.0000 99.2057 99.3690 99.4988 99.6019 99.6838 99.7488 99.8005 99.8415 99.8741 99.9000 99.9206 99.9369 99.9499 99.9602 99.9684 99.9749 99.9800 99.9842 99.9874 99.9900 
CPU times: user 16.5 ms, sys: 6.83 ms, total: 23.3 ms
Wall time: 23 ms


In [12]:
print("compute sample size in each bin")
N_prQ = sampleSizeAtAllRanks(targetranks,pr,ranks_ref,rank_locations=rank_locations)
print(np.nansum(N_prQ),pr.size)

compute sample size in each bin
2553.0 11520


In [13]:
%%time
print("compute O'Gorman&Schneider scaling with environmental temperature profile")
eps_OGS09, pr_scOGS09_vQ = computeScalingOGS09AtAllRanks(targetranks,omega,ta,pres,pr,
    ranks_ref=ranks_ref[iQ_slice],percentiles_ref=percentiles[iQ_slice],bins=bins,
                                                         rank_locations=rank_locations)
pr_scOGS09_vQ = adjustRanks(pr_scOGS09_vQ,targetranks,ranks_ref)
pr_scOGS09 = scalingOGS09(omega,ta,pres,efficiency=eps_OGS09,levdim=1)
print("eps_OGS09 =",eps_OGS09)

compute O'Gorman&Schneider scaling with environmental temperature profile
eps_OGS09 = 1.05358854943
CPU times: user 660 ms, sys: 118 ms, total: 779 ms
Wall time: 787 ms


In [14]:
%%time
print("compute O'Gorman&Schneider scaling with moist adiabat")
# Get temperature values at lowest level
levdim = 1
nlev = pres.shape[levdim]
i_bottom = bottomIndex(np.moveaxis(pres,levdim,-1).ravel()[-nlev:])
ta_bottom = np.take(ta,axis=levdim,indices=i_bottom)
# Compute scaling
eps_OGS09ad, pr_scOGS09ad_vQ = computeScalingOGS09AtAllRanks(targetranks,omega,ta_bottom,pres,pr,
    temp_type='adiabat',relhum=relhum,ranks_ref=ranks_ref[iQ_slice],percentiles_ref=percentiles[iQ_slice],bins=bins,
                                                         rank_locations=rank_locations)
pr_scOGS09ad_vQ = adjustRanks(pr_scOGS09ad_vQ,targetranks,ranks_ref)
pr_scOGS09ad = scalingOGS09(omega,ta_bottom,pres,temp_type='adiabat',relhum=relhum,efficiency=eps_OGS09ad,levdim=1)
print("eps_OGS09ad =",eps_OGS09ad)

compute O'Gorman&Schneider scaling with moist adiabat
eps_OGS09ad = 1.09914327972
CPU times: user 880 ms, sys: 81.5 ms, total: 961 ms
Wall time: 1.03 s


## Compute statistics

### 1D PDFs

In [15]:
print()
print("-- compute statistics")
print()


-- compute statistics



In [16]:
# %%time
print("mean of scalings in pr bins")
pr_scOGS09_prQ = meanXAtAllYRanks(targetranks,pr_scOGS09,pr,ranks_ref,rank_locations=rank_locations)
pr_scOGS09ad_prQ = meanXAtAllYRanks(targetranks,pr_scOGS09ad,pr,ranks_ref,rank_locations=rank_locations)

mean of scalings in pr bins


In [17]:
%%time
print("interquartile and 90% ranges")
ranks_I90 = (5,95)
ranks_IQR = (25,75)

pr_scOGS09_I90 = XPercentilesAtAllYRanks(targetranks,pr_scOGS09,ranks_I90,pr,ranks_ref,rank_locations_X=rank_locations)
pr_scOGS09_IQR = XPercentilesAtAllYRanks(targetranks,pr_scOGS09,ranks_IQR,pr,ranks_ref,rank_locations_X=rank_locations)

pr_scOGS09ad_I90 = XPercentilesAtAllYRanks(targetranks,pr_scOGS09ad,ranks_I90,pr,ranks_ref,rank_locations_X=rank_locations)
pr_scOGS09ad_IQR = XPercentilesAtAllYRanks(targetranks,pr_scOGS09ad,ranks_IQR,pr,ranks_ref,rank_locations_X=rank_locations)


interquartile and 90% ranges
CPU times: user 18.7 ms, sys: 1.16 ms, total: 19.8 ms
Wall time: 19.9 ms


### Decompose variance into its contributions from qvstar and omega

In [18]:
print()
print("-- decompose variance into qvstar and omega contributions")
print()


-- decompose variance into qvstar and omega contributions



In [19]:
%%time 
print("Compute the variance in pr")
scaling_names = 'scOGS09','scOGS09ad'

Ntimes = 100 if bootstrap else 1
fsub = 0.5 if bootstrap else 1

for sc_name in scaling_names:

    # compute variance from full sample
    pr_values = getattr(thismodule,'pr_%s'%sc_name)
    var_pr = varXAtAllYRanks(targetranks,pr_values,
                            pr,ranks_ref,rank_locations=rank_locations)
    setattr(thismodule,'var_pr_%s_prQ'%sc_name,var_pr)
    
    # bootstrapping
    all_var_pr = np.empty((Ntimes,var_pr.size))
    for i in range(Ntimes):
        all_var_pr[i,:] = varXAtAllYRanks(targetranks,pr_values,
                            pr,ranks_ref,rank_locations=rank_locations,random_fraction=fsub)
    if bootstrap:
        setattr(thismodule,'var_pr_%s_I25_prQ'%sc_name,all_var_pr[0])
        setattr(thismodule,'var_pr_%s_I75_prQ'%sc_name,all_var_pr[1])

Compute the variance in pr
CPU times: user 1.05 s, sys: 8.85 ms, total: 1.06 s
Wall time: 1.07 s


In [20]:
print("normalized variability")
for sc_name in scaling_names:
    norm_var_pr = getattr(thismodule,'var_pr_%s_prQ'%sc_name)/getattr(thismodule,'pr_%s_prQ'%sc_name)**2
    setattr(thismodule,'norm_var_pr_%s_prQ'%sc_name,norm_var_pr)
    if bootstrap:
        for int_bnd in 'I25','I75':
            norm_var_pr_Ibnd = getattr(thismodule,'var_pr_%s_%s_prQ'%(sc_name,int_bnd))/\
                getattr(thismodule,'pr_%s_prQ'%sc_name)**2
            setattr(thismodule,'norm_var_pr_%s_%s_prQ'%(sc_name,int_bnd),norm_var_pr_Ibnd)
            # for printing only:
            setattr(thismodule,'norm_var_pr_%s'%int_bnd,norm_var_pr_Ibnd)
    
    print("scaling %s : %1.5f"%(sc_name,np.nanmean(norm_var_pr[iQ_slice])),end='')
    if bootstrap:
        print("\t (%1.5f,%1.5f)"%(np.nanmean(norm_var_pr_I25[iQ_slice]),
                                  np.nanmean(norm_var_pr_I75[iQ_slice])))
    else:
        print()

normalized variability
scaling scOGS09 : 0.01028	 (0.01098,0.00627)
scaling scOGS09ad : 0.01949	 (0.01197,0.00731)


In [121]:
print()
print("- Compute EOF decomposition and contribution terms -")
print()

def contributionsVarianceProduct(X,Y):
    
    """X and Y are 1d arrays."""
    var_x = np.nanvar(X)
    var_y = np.nanvar(Y)
    cov_term = np.nanmean((X**2-np.nanmean((X**2)))*(Y**2-np.nanmean(Y**2))) - \
        2*np.nanmean(X)*np.nanmean(Y)*np.nanmean((X-np.nanmean(X))*(Y-np.nanmean(Y)))

    return var_x,var_y,cov_term

def getEOFmodes(A,n_max = None,threshold=0.95):
    
    """levdim is 0"""
    
    U,S,Vh = np.linalg.svd(A.T)
    if n_max is None:
        n_max = np.argmax(np.cumsum((S**2)/(S**2).sum())>threshold)+1
    phi = np.nan*np.empty((n_max,Vh.shape[1]))
    lambdas = np.nan*np.empty((n_max,))
    coefs = np.nan*np.empty((n_max,U.shape[0]))
    for i in range(min(n_max,len(S))):
        phi[i] = Vh[i,:]
        lambdas[i] = S[i]
        coefs[i] = S[i]*U[:,i]
    lambda_2_sum = (S**2).sum()
    return np.array(coefs),np.array(lambdas),np.array(phi),lambda_2_sum

def computeGramMatrixOfModes(qvstar_modes,omega_modes,pres,levdim=0):
    
    NX,NY = qvstar_modes.shape[0], omega_modes.shape[0]
    
    gram = np.empty((NX,NY))
    for i in range(NX):
        for j in range(NY):
            
            omega_mode_mid = np.apply_along_axis(arr=omega_modes[j],axis=levdim,
                                             func1d=(lambda x:np.convolve(x,(0.5,0.5),mode='valid')))
            q_mode_diff = np.diff(qvstar_modes[i],axis=levdim)
            gram[i,j] = np.sum(omega_mode_mid*q_mode_diff)/gg
            
    return gram

n_max_qvstar = 3
n_max_omega = 4
n_modes = n_max_qvstar*n_max_omega
contribQOmega_varOGS09_varQ = np.nan*np.empty((n_modes,len(ranks_ref)))
contribQOmega_varOGS09_varOmega = np.nan*np.empty((n_modes,len(ranks_ref)))
contribQOmega_varOGS09_covQOmega = np.nan*np.empty((n_modes,len(ranks_ref)))

for rank in targetranks:
    
    rankid = rankID(rank)

    #- Get profiles and fluctuations at given percentile
    stencil_Q = rank_locations[rankid]
    if stencil_Q.sum() == 0:
        continue
    omega_prof_Q = sampleFlattened(omega,stencil_Q)
    qvstar_prof_Q = sampleFlattened(qvstar,stencil_Q)
    pres_Q = sampleFlattened(pres,stencil_Q)
    m_omega_prof_Q = np.mean(omega_prof_Q,axis=1)
    m_qvstar_prof_Q = np.mean(qvstar_prof_Q,axis=1)
    m_pres = np.mean(pres_Q,axis=1)
    prime_omega_prof_Q = np.apply_along_axis(lambda x: x-m_omega_prof_Q,axis=0,arr=omega_prof_Q)
    prime_qvstar_prof_Q = np.apply_along_axis(lambda x: x-m_qvstar_prof_Q,axis=0,arr=qvstar_prof_Q)

    #- EOF decomposition
    c_qvstar, lambda_qvstar, phi_qvstar,lambda_2_sum_qvstar = getEOFmodes(prime_qvstar_prof_Q,n_max=n_max_qvstar)
    c_omega, lambda_omega, phi_omega, lambda_2_sum_omega = getEOFmodes(prime_omega_prof_Q,n_max=n_max_omega)

    #- Gram matrix of modes (vertical scaling on each pair of modes)
    # first dimension are thermodynamic modes, second are dynamic
    beta_Q = computeGramMatrixOfModes(phi_qvstar,phi_omega,m_pres)
    omega_mid = np.apply_along_axis(arr=omega_prof_Q,axis=0,
                                func1d=(lambda x:np.convolve(x,(0.5,0.5),mode='valid')))
    qvstar_diff = np.diff(qvstar_prof_Q,axis=0)
    beta_0 = np.sum(omega_mid*qvstar_diff)/gg

    #- Compute all contribution terms - store as a new dimension in an array of size dim(beta),#contribs
    contribs = np.empty(beta_Q.shape+(3,))
    for i,j in np.ndindex(*(beta_Q.shape)):
        contribs[i,j,:] = (beta_Q[i,j]/beta_0)**2*np.array(contributionsVarianceProduct(c_qvstar[i,:]*lambda_qvstar[i],
                                                                               c_omega[i,:]*lambda_omega[i]))
    contribs_prQ = np.vstack(list(contribs))
    
    #- Store results in array
    iQ = indexOfRank(rank,ranks_ref)
    contribQOmega_varOGS09_varQ[:,iQ] = contribs_prQ[:,0]
    contribQOmega_varOGS09_varOmega[:,iQ] = contribs_prQ[:,1]
    contribQOmega_varOGS09_covQOmega[:,iQ] = contribs_prQ[:,2]


- Compute EOF decomposition and contribution terms -

